In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!/home/ektov1-av_ca-sbrf-ru/bin/python35
import os
import sys
curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))


# sys.path.insert(0, './../src')
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

#import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(filename='./__move_dbs__.log',level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)


import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
import decimal
import datetime
from getpass import getpass

import jaydebeapi

from transliterate import translit

from spark_connector import SparkConnector
from sparkdb_loader import spark
from connector import OracleDB, TeraDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.utils import AnalysisException

from pyspark.sql import Row

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
import pandas as pd

# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))


##-----------------------------------------
conn_schema = 'sbx_team_digitcamp' #'dl_t_team_ds_kb_sme' 'dl_t_team_speech_analytics'
new_schema = 'dl_team_digitcamp'
ga_schema = 'sklod_external_google_analytics'
##-----------------------------------------


def show(self, n=10):
    return self.limit(n).toPandas()

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

def essense(channel: str, prod_cd: str):
    message = "{}: {} retargeting".format(channel, prod_cd)
    return message

essense_udf = f.udf(essense, StringType())

pyspark.sql.dataframe.DataFrame.show = show

def print_and_log(message: str):
    print(message)
    logger.info(message)
    return None


# Start Spark Context with Hive Support

In [3]:
print("### Starting spark context. Run!")

sp = spark(schema=conn_schema,
           dynamic_alloc=False,
           kerberos_auth=False,
           numofinstances=4, 
           numofcores=8,
           executor_memory='20g', 
           driver_memory='25g'
           )
hive = sp.sql

print(sp.sc.version)

### Starting spark context. Run!
====================================================== 2022-02-25 ======================================================
# __init__ : begin
2.4.0.cloudera2


In [16]:
# hive.setConf("mapred.map.max.attempts", 100)
# hive.setConf("mapred.reduce.max.attempts", 100)
# hive.setConf("mapred.skip.map.max.skip.records", 10000)
# hive.setConf("mapred.skip.attempts.to.start.skipping", 1)
# hive.setConf("mapred.map.max.attempts", 100)

In [4]:
sp.sc.getConf().getAll()

[('spark.sql.hive.convertMetastoreParquet', 'false'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.sql.parquet.binaryAsString', 'true'),
 ('spark.driver.host', 'pklis-chd001986.labiac.df.sbrf.ru'),
 ('spark.task.maxFailures', '15'),
 ('spark.sql.queryExecutionListeners',
  'com.cloudera.spark.lineage.NavigatorQueryListener'),
 ('spark.sql.autoBroadcastJoinThreshold', '20971520'),
 ('spark.driver.memory', '15g'),
 ('spark.sql.shuffle.partitions', '400'),
 ('spark.eventLog.dir', 'hdfs://nsld3/user/spark/spark2ApplicationHistory'),
 ('spark.ui.port', '4445'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.yarn.secondary.jars', 'terajdbc4.jar,tdgssconfig.jar'),
 ('spark.hadoop.hive.exec.max.dynamic.partitions', '50000'),
 ('spark.executor.memoryOverhead', '8g'),
 ('spark.sql.hive.metastore.jars',
  '${env:HADOOP_COMMON_HOME}/../hive/lib/*:${env:HADOOP_COMMON_HOME}/client/*'),
 ('spark.lineage.log.dir', '/var/log/spark2/lineage'),
 ('spark.blacklist.task.maxT

In [4]:
sdf = hive.table("sbx_team_digitcamp.ma_mmb_product_dict")

In [8]:
hive.sql("SHOW TBLPROPERTIES {}.{}".format(conn_schema, 'ma_mmb_product_dict')).collect()

[Row(key='transient_lastDdlTime', value='1593439429')]

In [11]:
hive.sql("DROP TABLE IF EXISTS TMP_TBL_TEST PURGE")
hive.sql("CREATE TABLE TMP_TBL_TEST (INN STRING, SCORE FLOAT)")
hive.sql("ALTER TABLE TMP_TBL_TEST SET TBLPROPERTIES ('auto.purge'='true')")

DataFrame[]

In [16]:
sdf.write \
   .format('parquet') \
   .mode("append")\
   .option("path", "hdfs://clsklsbx/user/team/team_cvm/hive/").saveAsTable("tmp_ma_mmb_product_dict_2")

## Initialize Conection to TDDB

In [3]:
TERADATA_HOST = "TDSB15.cgs.sbrf.ru"
DB = "PRD_DB_CLIENT4D_DEV1"
# DATABASE_NAME = "report_all_voronka"
USERNAME = "ektov1-av"
PASSWORD = getpass()
db = TeraDB(TERADATA_HOST, DB, USERNAME, PASSWORD)

········


## Check actual User' Grants on Specified DatabaseName

In [5]:
sql =\
'''(
    SELECT
    A.ROLENAME,
    A.GRANTEE AS USER_ID,
    A.GRANTOR AS ADMIN_ID,
    B.DATABASENAME,
    B.TABLENAME,
    B.GRANTORNAME,
    B.ACCESSRIGHT
    FROM    DBC.ROLEMEMBERS A
    JOIN    DBC.ALLROLERIGHTS B
    ON  A.ROLENAME = B.ROLENAME
    WHERE  GRANTEE='{}' AND DATABASENAME like '%_LIENT4D_DEV1%'
    GROUP BY 1,2,3,4,5,6,7
    --ORDER BY 2,1,6
    ) as t
'''.format(USERNAME)
sdf = sp.get_teradata(db, sql)

df = sdf.toPandas()

In [6]:
df

,RoleName,USER_ID,ADMIN_ID,DatabaseName,TableName,GrantorName,AccessRight
0,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,R
1,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,D
2,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,CF
3,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,AP
4,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,PC
5,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,CV
6,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,PE
7,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,AF
8,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,I
9,PRD_RL_CLIENT4D_DEV1_Adm ...,Ektov1-AV ...,tsa ...,PRD_DB_CLIENT4D_DEV1 ...,All ...,tsa ...,DT


In [101]:
my_sql = '''(SELECT 
                 *    
             FROM dbc.columnsV 
             WHERE DatabaseName ='{}') as t
         '''.format('PRD_DB_CLIENT4D_DEV1')

df = sp.get_teradata(db, my_sql)

In [102]:
df.show(40)

,DatabaseName,TableName,ColumnName,ColumnFormat,ColumnTitle,SPParameterType,ColumnType,ColumnUDTName,ColumnLength,DefaultValue,...,ColumnPartitionFormat,ColumnPartitionAC,PseudoUDTFieldId,PseudoUDTFieldType,StorageFormat,DatasetSchemaName,InlineLength,TSColumnType,AutoColumn,RowVersionNo
0,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,partner_nm,X(100),None,,CV,None,200,None,...,NA,NA,None,None,None,None,None,None,None,None
1,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,SOURCE_LEAD_ID,X(15),None,,CV,None,30,None,...,NA,NA,None,None,None,None,None,None,None,None
2,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,info,X(1000),None,,CV,None,4096,None,...,NA,NA,None,None,None,None,None,None,None,None
3,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,flag_no_dozvon,-(10)9,None,,I,None,4,None,...,NA,NA,None,None,None,None,None,None,None,None
4,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,FEE,--(15).9(7),None,,D,None,16,None,...,NA,NA,None,None,None,None,None,None,None,None
5,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,flag_dozvon,-(3)9,None,,I1,None,1,None,...,NA,NA,None,None,None,None,None,None,None,None
6,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,ckp_id,X(100),None,,CV,None,200,None,...,NA,NA,None,None,None,None,None,None,None,None
7,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,Company_id,X(15),None,,CV,None,30,None,...,NA,NA,None,None,None,None,None,None,None,None
8,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,FEE_PERCENT,--(15).9(7),None,,D,None,16,None,...,NA,NA,None,None,None,None,None,None,None,None
9,PRD_DB_CLIENT4D_DEV1,s_cvm_Import_DIM_ProductOffer_ProductOffer_upd...,RATE,--(15).9(7),None,,D,None,16,None,...,NA,NA,None,None,None,None,None,None,None,None


## Handling Tera Aborted Transactions

In [ ]:
'''
java.sql.BatchUpdateException: 
[Teradata JDBC Driver] [TeraJDBC 15.10.00.37] [Error 1338] [SQLState HY000] 
A failure occurred while executing a PreparedStatement batch request. 
Details of the failure can be found in the exception chain that is accessible with getNextException.
'''

In [52]:
my_sql = '''(
               SELECT TOP 100 PERCENT
                 *
                 --UserName, ErrorCode, ErrorText, StartTime     
               FROM dbc.DBQLogTbl 
               WHERE (lower(UserName) like '%ektov%') and
                     (ErrorText like '%Transaction ABORTed due to deadlock%')
               ORDER BY StartTime DESC
            ) as Q
         '''

df = sp.get_teradata(db, my_sql)

In [53]:
df.take(1)

[Row(ProcID=Decimal('30679'), CollectTimeStamp=datetime.datetime(2022, 2, 17, 3, 23, 17, 140000), QueryID=Decimal('306790575673127150'), ZoneId=bytearray(b'\x00\x00\x00\x00'), UserID=bytearray(b'\x00\x00\x14['), AcctString='DBC', ExpandAcctString='DBC', SessionID=6634959, LogicalHostID=1, RequestNum=7, InternalRequestNum=7, LogonDateTime=datetime.datetime(2022, 2, 17, 3, 18, 0, 460000), AcctStringTime=None, AcctStringHour=None, AcctStringDate=None, LogonSource='(TCP/IP) e33c 10.110.10.143 TDSB15.CGS.SBRF.RU;TDSB15COP29. CID=62C22F3 EKTOV1-AV_CA-SBRF-RU JDBC15.10.00.37;1.8.0_312 01 LSS                                                                                                                                   ', AppID='JDBC15.10.00.37;1.8.0_312                                                                 ', ClientID='EKTOV1-AV_CA-SBRF-RU                                                                      ', ClientAddr='10.110.10.143                                               

# Fetch some data from Hive Table

### Get actual partitions from MetaStore

In [5]:
from itertools import cycle

In [6]:
conn_schema = 'sbx_team_digitcamp' #'sbx_t_team_cvm'
table_name = 'ma_cmdm_ma_deal_new' #'lal_db_hist_in'

In [ ]:
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
descr = hive.sql("describe extended {}.{}".format(conn_schema,table_name)).collect()
push_down = True
loop_rows = cycle(descr)
nextelem = next(loop_rows)
while push_down:
    thiselemen, nextelem = nextelem, next(loop_rows)
    if nextelem.asDict()['col_name'] =='# Partition Information':
        next(loop_rows)
        part_col = next(loop_rows)
        part_col_info = part_col.asDict()
        push_down = False
        
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [7]:
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
part_name = part_col_info['col_name']
# set the fix number of sorted patitions to catch from metastore
numdays = 10       
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

hasPartitioned = len([item.asDict() for item in descr if item.asDict()['col_name'] =='# Partition Information']) > 0
if hasPartitioned:
    try:
        parts = hive.sql("show partitions {}.{}".format(conn_schema,table_name)).collect()
        parts = [part['partition'] for part in parts if not part['partition'].endswith('__HIVE_DEFAULT_PARTITION__')]
        parts = sorted(parts,reverse=True)
        max_part = parts[0]
        extract_date=re.compile("\d{4}\-\d{2}\-\d{2}")
        ext = extract_date.search(max_part)
        try:
            max_trunc_dt = ext.group(0)
        except:
            max_trunc_dt = None
        if part_col_info['data_type'] in ('date', 'timestamp', 'string') and (max_trunc_dt is not None): 
            parts = sorted(parts, reverse=True, key=lambda x: 
                           datetime.datetime.strptime(x.split(part_name+'=')[-1].split('/')[0], '%Y-%m-%d'))
        else:
            parts = sorted(parts, reverse=True, key=lambda x: int(x.split(part_name+'=')[-1].split('/')[0]))
        last_part_lst = sorted(list(set([part.split(part_name+'=')[-1].split('/')[0] for part in parts])), 
                               reverse=True)[:numdays]
    except (AnalysisException, IndexError):
        last_part_lst = None
else:
    last_part_lst = None
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [8]:
last_part_lst

['2022-02-01', '2022-01-01']

## Filter Corrupted Partitions

In [9]:
HDFS_PATH = "hdfs://clsklsbx/user/team/team_digitcamp/hive/{}/{}".format(table_name, part_name)+"={}/"
noncorrupCTL = []
for ctl in last_part_lst:
    try:
        sdf = hive.read.load(path=HDFS_PATH.format(ctl), format='parquet').limit(10)
        noncorrupCTL.append(ctl)
    except (AnalysisException) as err:
        if "Unable to infer schema for Parquet" in str(err):
            print(" Unable to infer schema for Parquet. It must be specified manually")
            
noncorrupCTL

['2022-02-01', '2022-01-01']

## Select Appropriate Rows Under User Conditions

In [ ]:
sdf = hive.table("{}.{}".format(conn_schema,table_name))

lstofdates = [datetime.datetime.strptime(item, '%Y-%m-%d') for item in noncorrupCTL]
sdf = sdf.filter(sdf.CREATE_DT_DAY.isin(lstofdates))

### Or via pure SQL

In [10]:
sdf = hive.sql('''select * from {}.{} 
                  where {} between '{}' and '{}' '''.format(conn_schema,
                                                            table_name,
                                                            part_name.lower(),
                                                            noncorrupCTL[-1],
                                                            noncorrupCTL[0]
                                                           )
              )

In [11]:
sdf = sdf.withColumn("row_id", f.monotonically_increasing_id().cast(IntegerType()))

In [14]:
# df = sdf.sample(0.01).toPandas()
# df.to_csv("~/ma_deals.csv", sep=",", encoding="cp1251", index=False)

# with open("/home/ektov1-av_ca-sbrf-ru/ma_deals.csv", "r", encoding="cp1251") as fin, 
#      open("/home/ektov1-av_ca-sbrf-ru/ma_deals_utf.csv", "w", encoding="utf8") as fout:
#     _in = csv.reader(fin)
#     _out = csv.writer(fout)
#     for ind, row in enumerate(_in):
#         row = [el.encode("utf8").decode("utf8") for el in row]
#         _out.writerow(row)

In [12]:
sdf.cache()

DataFrame[INN: string, KPP: string, HOST_APPL_ID: string, HOST_CRTR_EMP_ID: string, HOST_VKS_EMP_ID: string, HOST_CUST_ORG_ID: string, APPL_SALE_MTHD_TYPE_NAME: string, CREATE_DT: timestamp, PLAN_COMPLETE_DT: timestamp, COMPLETE_DT: timestamp, HOST_PROD_ID: string, PROD_TYPE_NAME: string, APPL_TYPE_NAME: string, APPL_CORE_TXT: string, SUM_RUB: decimal(38,10), MARGIN_PCT: decimal(38,10), GROSS_INC_RUB: decimal(38,10), CLOSE_REASON_TYPE_NAME: string, HOST_CM_EMP_ID: string, APPL_STAGE_NAME: string, ARCHIVE_FLG: string, ARCHIVE_DT: timestamp, HOST_LEAD_ID: string, REQUEST_SUM_VAL: decimal(38,10), crm_active_flg: bigint, product_full_nm: string, product_short_nm: string, product_cd_mmb: string, product_subgroup: string, product_type: string, product_priority: bigint, channel_sale: bigint, channel_eco_expert: bigint, channel_mmb_ms: bigint, channel_mmb_ckr: bigint, channel_mmb_mkk: bigint, channel_dzo: bigint, creation_month: string, product_group: string, row_id: int]

In [13]:
sdf.count()

1127096

In [ ]:
sdf.show()

# IF HIVE Partitions are corrupted then try to restore data from Oracle Replica

## FETCH Consistent Replica's Rows from Oracle

In [5]:
sql = """
(
select /*+ parallel (4) */ * from ISKRA_CVM.ga_all_scenarios_hist
where load_dt > to_timestamp('{}', 'yyyy-mm-dd')
)""".format('2022-01-01')
    
sdf = sp.get_oracle(OracleDB('iskra4'), sql)

In [51]:
sdf.dtypes

[('INN', 'string'),
 ('KPP', 'string'),
 ('HOST_APPL_ID', 'string'),
 ('HOST_CRTR_EMP_ID', 'string'),
 ('HOST_VKS_EMP_ID', 'string'),
 ('HOST_CUST_ORG_ID', 'string'),
 ('APPL_SALE_MTHD_TYPE_NAME', 'string'),
 ('CREATE_DT', 'timestamp'),
 ('PLAN_COMPLETE_DT', 'timestamp'),
 ('COMPLETE_DT', 'timestamp'),
 ('HOST_PROD_ID', 'string'),
 ('PROD_TYPE_NAME', 'string'),
 ('APPL_TYPE_NAME', 'string'),
 ('APPL_CORE_TXT', 'string'),
 ('SUM_RUB', 'decimal(38,10)'),
 ('MARGIN_PCT', 'decimal(38,10)'),
 ('GROSS_INC_RUB', 'decimal(38,10)'),
 ('CLOSE_REASON_TYPE_NAME', 'string'),
 ('HOST_CM_EMP_ID', 'string'),
 ('APPL_STAGE_NAME', 'string'),
 ('ARCHIVE_FLG', 'string'),
 ('ARCHIVE_DT', 'timestamp'),
 ('HOST_LEAD_ID', 'string'),
 ('REQUEST_SUM_VAL', 'decimal(38,10)'),
 ('crm_active_flg', 'bigint'),
 ('product_full_nm', 'string'),
 ('product_short_nm', 'string'),
 ('product_cd_mmb', 'string'),
 ('product_subgroup', 'string'),
 ('product_type', 'string'),
 ('product_priority', 'bigint'),
 ('channel_sale', '

## Temporarily Copy fetched data to HIVE

In [16]:
sdf = sdf.select([col for col in sdf.columns if col not in ("COMMONSEGMENTOUID")] + ["COMMONSEGMENTOUID"])

In [18]:
sdf = sdf.select([col.upper() for col in sdf.columns])
        
cols = ', '.join([col + ' ' + type.upper() for col, type in sdf.dtypes])
cols

'INN STRING, SBBOLUSERID STRING, ID_CRM STRING, SCENARIO_ID STRING, CHANNEL STRING, ID_PRODUCT STRING, MADEAL_PRODUCT_NORM STRING, MADEAL_PROD_TYPE_NAME STRING, MADEAL_APPL_CORE_TXT STRING, MADEAL_COMPLETE_DT TIMESTAMP, HASPRODUCT DECIMAL(38,0), ESSENSE STRING, RETURNCNT DECIMAL(38,0), NUMBEROFSTEPS DECIMAL(38,0), FUNNEL_RATE STRING, SUM_OFFER_PRIORITY DECIMAL(38,0), SUM_OFFER_NONTOP DECIMAL(38,0), MINSESSIONSTARTTIME TIMESTAMP, MAXSESSIONSTARTTIME TIMESTAMP, LOAD_DT TIMESTAMP, CTL_LOADING DECIMAL(38,0), COMMONSEGMENTOUID STRING'

In [20]:
hive_table_name   = "GA_ALL_SCENARIOS_PART_NEW"
tmp_tbl_name = "tmp_ora2hive_dlg"

sdf.registerTempTable(tmp_tbl_name)

hive.sql("CREATE TABLE {}.{} ({})".format(conn_schema, table_name, cols))
hive.sql("INSERT INTO {}.{} SELECT * FROM {}".format(conn_schema, hive_table_name, tmp_tbl_name))

DataFrame[]

# Fetch data from saved HIVE TBL

In [68]:
# conn_schema = 'sbx_t_team_cvm'
# hive_table_name   = "LAL_DB_HIST_IN"
# sdf = hive.table("{}.{}".format(conn_schema, hive_table_name))

In [ ]:
# sdf.selectExpr("max(create_dt_day) as max_create_dt_day").show()

## Map ColumnTypes from Spark to Teradata

In [14]:
df = sdf.limit(1000).toPandas()

In [ ]:
df.dtypes

In [3]:
#**************************************************************************************************************
#**************************************************************************************************************
## Place Row_ID on the first column-position in your database. 
## Teradata always use the first column as Primary Index if not explicitly specified in 
## `INSERT INTO SELECT * FROM ` clause
#**************************************************************************************************************
#**************************************************************************************************************

In [16]:
cols_specific = {}
cols_specific["row_id"] = ['INTEGER', 'INT']
#cols_specific {COL_NAME: [TERATYPE, HIVE_TYPE]}

isDateFmt = False
PRIMARY_INDX = 'ROW_ID'

str_ = '''CREATE MULTISET TABLE {0}, NO FALLBACK, 
                                     NO BEFORE JOURNAL, NO AFTER JOURNAL, 
                                     CHECKSUM = DEFAULT, 
                                     DEFAULT MERGEBLOCKRATIO ( ROW_ID INTEGER, '''
for column_name, column in df.iteritems():
    try:
        if isinstance(column[column.first_valid_index()], str):
            if (df[column_name].str.len().max() >= 4000) or ('COMMONSEGMENTOUID' in column_name.upper()):
    #             df.drop(columns=[column_name], inplace=True)
                str_+=column_name.upper() + ' ' +'CLOB, '
                cols_specific[column_name] = ['CLOB', 'STRING']
            else:
                if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
                    str_+=column_name.upper() + ' ' +'VARCHAR(20), '
                    cols_specific[column_name] = ['VARCHAR(20)', 'STRING']
                elif part_name.upper() in column_name.upper():
                    str_+=column_name.upper() + ' ' +'DATE, '
                    cols_specific[column_name] = ['DATE', 'DATE_FORMAT({}, "yyyy-MM-dd")']
                else:
                    str_+=column_name.upper() + ' ' +'VARCHAR(800), '
                    cols_specific[column_name] = ['VARCHAR(800)', 'STRING']
        elif isinstance(column[column.first_valid_index()], np.integer) and (column_name.upper() != "ROW_ID"):
            if len(str(column[column.first_valid_index()])) < 1.e6:
                str_+=column_name.upper() + ' ' +'INTEGER, '
                cols_specific[column_name] =['INTEGER', 'INT']
            else:
                str_+=column_name.upper() + ' ' +'INTEGER, '
                cols_specific[column_name] =['BIGINT', 'BIGINT']            
        elif (
               ( isinstance(column[column.first_valid_index()], decimal.Decimal) ) or 
               ( isinstance(column[column.first_valid_index()], float) )
             ):
    #         df[column_name] = df[column_name].fillna(0.0)
            str_+=column_name.upper() + ' ' +'FLOAT, ' 
            cols_specific[column_name] = ['FLOAT', 'FLOAT']
        elif (
                isinstance(column[column.first_valid_index()], pd.Timestamp) or 
                isinstance(column[column.first_valid_index()], datetime.date)
             ):
            if hasattr(column[column.first_valid_index()],'minute'):
                if column[column.first_valid_index()].minute == 0:
                    isDateFmt = True
                else:
                    isDateFmt = False
                    str_+=column_name.upper() + ' ' +"TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS', "
                    cols_specific[column_name] = ['TIMESTAMP', 'DATE_FORMAT({}, "yyyy-MM-dd HH:mm:ss")']
            else:
                isDateFmt = True
            if isDateFmt:
                str_+=column_name.upper() + ' ' +"DATE FORMAT 'YYYY-MM-DD', "
                cols_specific[column_name] = ['DATE', 'DATE_FORMAT({}, "yyyy-MM-dd")']            
        else:
            None  
    except:
        str_+=column_name.upper() + ' ' +'VARCHAR(800), '
        cols_specific[column_name] = ['VARCHAR(800)', 'STRING']
        
res=str_.strip()[:-1] + ' )' + ' PRIMARY INDEX ({});'.format(PRIMARY_INDX)

cr_tbl_sql = res

print(cr_tbl_sql)

CREATE MULTISET TABLE {0}, NO FALLBACK, 
                                     NO BEFORE JOURNAL, NO AFTER JOURNAL, 
                                     CHECKSUM = DEFAULT, 
                                     DEFAULT MERGEBLOCKRATIO ( ROW_ID INTEGER, INN VARCHAR(20), KPP VARCHAR(20), HOST_APPL_ID VARCHAR(800), HOST_CRTR_EMP_ID VARCHAR(800), HOST_VKS_EMP_ID VARCHAR(800), HOST_CUST_ORG_ID VARCHAR(800), APPL_SALE_MTHD_TYPE_NAME VARCHAR(800), CREATE_DT TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS', PLAN_COMPLETE_DT DATE FORMAT 'YYYY-MM-DD', COMPLETE_DT DATE FORMAT 'YYYY-MM-DD', HOST_PROD_ID VARCHAR(800), PROD_TYPE_NAME VARCHAR(800), APPL_TYPE_NAME VARCHAR(800), APPL_CORE_TXT VARCHAR(800), SUM_RUB FLOAT, MARGIN_PCT VARCHAR(800), GROSS_INC_RUB FLOAT, CLOSE_REASON_TYPE_NAME VARCHAR(800), HOST_CM_EMP_ID VARCHAR(800), APPL_STAGE_NAME VARCHAR(800), ARCHIVE_FLG VARCHAR(800), ARCHIVE_DT VARCHAR(800), HOST_LEAD_ID VARCHAR(800), REQUEST_SUM_VAL VARCHAR(800), CRM_ACTIVE_FLG INTEGER, PRODUCT_FULL_NM VARCHA

## Set a first Connection to Teradata Host via jaydebeapi

In [17]:
JDBC_ARGUMENTS = "LOGMECH=LDAP, CHARSET=UTF8,TYPE=FASTEXPORT, COLUMN_NAME=ON, MAYBENULL=ON"
#DB = 'PRD_DB_CLIENT4D_DEV1'
conn = jaydebeapi.connect(
    jclassname="com.teradata.jdbc.TeraDriver",
    url="jdbc:teradata://{}/database={} , {}".format(TERADATA_HOST, DB, JDBC_ARGUMENTS),
    driver_args={"user": USERNAME, "password": PASSWORD},
    jars=['/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/tdgssconfig.jar',
          '/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/terajdbc4.jar']
)
curs = conn.cursor()

## Create an Empty Table with Predefined Column Types

In [18]:
tera_table_name = "MA_CMDM_MA_DEAL" #"LAL_DB_HIST_IN_NONPART"
tera_target_table_name = "MA_CMDM_MA_DEAL" #"LAL_DB_HIST_IN"

In [ ]:
curs.execute('DELETE FROM {} ALL'.format(tera_table_name))
curs.execute('DROP TABLE {}'.format(tera_table_name))

### Create NON Partitioned TBL

In [37]:
curs.execute(cr_tbl_sql.format(tera_table_name))
# curs.execute(cr_tbl_sql.split("PARTITION BY")[0].strip().format(tera_table_name))

In [34]:
curs.execute("COLLECT STAT ON {0} COLUMN(ROW_ID);".format(tera_table_name))

### Create Target Partitioned TBL

In [19]:
curs.execute('DELETE FROM {} ALL'.format(tera_target_table_name))
curs.execute('DROP TABLE {}'.format(tera_target_table_name))

In [20]:
PART_COL_NAME = "CREATION_MONTH"
cr_part_tbl_query = cr_tbl_sql.format(tera_target_table_name)[:-1] + \
" PARTITION BY RANGE_N({} BETWEEN DATE '2020-01-01' AND DATE '2025-01-01' EACH INTERVAL '1' DAY);".format(PART_COL_NAME)

curs.execute(cr_part_tbl_query)
# raw_data = curs.fetchall()

In [21]:
curs.execute("COLLECT STAT ON {0} COLUMN(ROW_ID);".format(tera_target_table_name))

In [22]:
conn.close()

# Inset Data From Hive Table into Teradata Table of Interest

## HIVE REMAPPING

In [25]:
tmp_tbl_name = "tmp_ora2hive_dlg"
sdf.registerTempTable(tmp_tbl_name)

In [26]:
_str = '''SELECT '''
for col_name, spec in cols_specific.items(): 
    if '{' not in spec[1]:
        _str+='''CAST({col} AS {type}) {col}, '''.format(col=col_name, type=spec[1])
    else:
        _str+='''CAST({} as {}) {}, '''.format(spec[1].format(col_name), spec[0], col_name)
fin_query = _str.strip()[:-1] + " FROM {}".format(tmp_tbl_name)

fin_query

'SELECT CAST(SUM_RUB AS FLOAT) SUM_RUB, CAST(DATE_FORMAT(COMPLETE_DT, "yyyy-MM-dd") as DATE) COMPLETE_DT, CAST(HOST_LEAD_ID AS STRING) HOST_LEAD_ID, CAST(INN AS STRING) INN, CAST(APPL_SALE_MTHD_TYPE_NAME AS STRING) APPL_SALE_MTHD_TYPE_NAME, CAST(PROD_TYPE_NAME AS STRING) PROD_TYPE_NAME, CAST(channel_dzo AS INT) channel_dzo, CAST(APPL_STAGE_NAME AS STRING) APPL_STAGE_NAME, CAST(APPL_TYPE_NAME AS STRING) APPL_TYPE_NAME, CAST(CLOSE_REASON_TYPE_NAME AS STRING) CLOSE_REASON_TYPE_NAME, CAST(product_full_nm AS STRING) product_full_nm, CAST(KPP AS STRING) KPP, CAST(GROSS_INC_RUB AS FLOAT) GROSS_INC_RUB, CAST(DATE_FORMAT(PLAN_COMPLETE_DT, "yyyy-MM-dd") as DATE) PLAN_COMPLETE_DT, CAST(DATE_FORMAT(creation_month, "yyyy-MM-dd") as DATE) creation_month, CAST(HOST_CRTR_EMP_ID AS STRING) HOST_CRTR_EMP_ID, CAST(HOST_CUST_ORG_ID AS STRING) HOST_CUST_ORG_ID, CAST(channel_sale AS INT) channel_sale, CAST(product_short_nm AS STRING) product_short_nm, CAST(channel_mmb_ckr AS INT) channel_mmb_ckr, CAST(chann

In [27]:
sdf_casted = hive.sql(fin_query)
sdf_casted = sdf_casted.select( ["row_id"]+[col for col in sdf.columns if col not in ("row_id")])

# Prepare col Specification for jDBC Driver

In [28]:
sdf_toload = sdf_casted

In [29]:
cols = ', '.join([col.upper() + ' ' + cols_specific[col][0] for col in sdf_toload.columns])
cols

'ROW_ID INTEGER, INN VARCHAR(20), KPP VARCHAR(20), HOST_APPL_ID VARCHAR(800), HOST_CRTR_EMP_ID VARCHAR(800), HOST_VKS_EMP_ID VARCHAR(800), HOST_CUST_ORG_ID VARCHAR(800), APPL_SALE_MTHD_TYPE_NAME VARCHAR(800), CREATE_DT TIMESTAMP, PLAN_COMPLETE_DT DATE, COMPLETE_DT DATE, HOST_PROD_ID VARCHAR(800), PROD_TYPE_NAME VARCHAR(800), APPL_TYPE_NAME VARCHAR(800), APPL_CORE_TXT VARCHAR(800), SUM_RUB FLOAT, MARGIN_PCT VARCHAR(800), GROSS_INC_RUB FLOAT, CLOSE_REASON_TYPE_NAME VARCHAR(800), HOST_CM_EMP_ID VARCHAR(800), APPL_STAGE_NAME VARCHAR(800), ARCHIVE_FLG VARCHAR(800), ARCHIVE_DT VARCHAR(800), HOST_LEAD_ID VARCHAR(800), REQUEST_SUM_VAL VARCHAR(800), CRM_ACTIVE_FLG INTEGER, PRODUCT_FULL_NM VARCHAR(800), PRODUCT_SHORT_NM VARCHAR(800), PRODUCT_CD_MMB VARCHAR(800), PRODUCT_SUBGROUP VARCHAR(800), PRODUCT_TYPE VARCHAR(800), PRODUCT_PRIORITY INTEGER, CHANNEL_SALE INTEGER, CHANNEL_ECO_EXPERT INTEGER, CHANNEL_MMB_MS INTEGER, CHANNEL_MMB_CKR INTEGER, CHANNEL_MMB_MKK INTEGER, CHANNEL_DZO INTEGER, CREATION

In [30]:
sdf_toload = sdf_toload.select([col.upper() for col in sdf.columns])

### !!! Don't forget to create an empty table before with CLOB types followed by all non-clob columns !!!

In [31]:
db.dsn

'jdbc:teradata://TDSB15.cgs.sbrf.ru/database=PRD_DB_CLIENT4D_DEV1, LOGMECH=LDAP, charset=UTF8,TYPE=FASTEXPORT,                                                                      COLUMN_NAME=ON, MAYBENULL=ON'

In [32]:
sdf_toload.rdd.getNumPartitions()

49891

In [ ]:
# Don't use `overwrite` mode because jdbc driver will reset your CREATE TABLE instructions into default `CREATE SET TABLE`
# and INDEX will be the first column in your dataset

In [ ]:
mode = 'append'
sdf_toload.coalesce(6)\
    .write \
    .format('jdbc') \
    .mode(mode) \
    .option('url', db.dsn) \
    .option('user', db.user) \
    .option('password', db.password) \
    .option('dbtable', tera_target_table_name) \
    .option('createTableColumnTypes', cols)\
    .option('driver', 'com.teradata.jdbc.TeraDriver') \
    .save()

### Using Worker's Parallelization

In [ ]:
num_treads = 10

mode = 'append'
sdf_toload.coalesce(10)\
    .write \
    .format('jdbc') \
    .mode(mode) \
    .option('url', db.dsn) \
    .option('user', db.user) \
    .option('password', db.password) \
    .option('dbtable', tera_table_name) \
    .option('createTableColumnTypes', cols)\
    .option('driver', 'com.teradata.jdbc.TeraDriver') \
    .option('numPartitions',num_treads) \
    .save()

## Check Data written to Teratada DWH

In [3]:
TERADATA_HOST = "TDSB15.cgs.sbrf.ru"
DB = "PRD_DB_CLIENT4D_DEV1 "
USERNAME = "ektov1-av"
PASSWORD = getpass()
db = TeraDB(TERADATA_HOST, DB, USERNAME, PASSWORD)

········


In [4]:
tera_table_name = "MA_CMDM_MA_DEAL" #"LAL_DB_HIST_IN_NONPART"
tera_target_table_name = "MA_CMDM_MA_DEAL" #"LAL_DB_HIST_IN"

## Via pySpark

In [ ]:
my_sql = '''(SELECT TOP 100 
                 *    
             FROM PRD_DB_CLIENT4D_DEV1.{}
             WHERE CREATE_DT_DAY = '2021-10-18'
             ) as t
         '''.format(tera_target_table_name)

td_df = sp.get_teradata(db, my_sql)

In [ ]:
td_df.show()

## or using batch loader jaydebeapi

In [5]:
JDBC_ARGUMENTS = "LOGMECH=LDAP, CHARSET=UTF8,TYPE=FASTEXPORT, COLUMN_NAME=ON, MAYBENULL=ON"
#DB = 'PRD_DB_CLIENT4D_DEV1'
conn = jaydebeapi.connect(
    jclassname="com.teradata.jdbc.TeraDriver",
    url="jdbc:teradata://{}/database={} , {}".format(TERADATA_HOST, DB, JDBC_ARGUMENTS),
    driver_args={"user": USERNAME, "password": PASSWORD},
    jars=['/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/tdgssconfig.jar',
          '/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/terajdbc4.jar']
)
curs = conn.cursor()

In [10]:
my_sql = '''SELECT TOP 1 
                 *    
             FROM PRD_DB_CLIENT4D_DEV1.{}
             --WHERE CREATE_DT_DAY = '2021-10-18'
         '''.format(tera_target_table_name)

curs.execute(my_sql)
cols = [desc[0] for desc in curs.description]

In [ ]:
my_sql = '''SELECT TOP 100 
                 *    
             FROM PRD_DB_CLIENT4D_DEV1.{}
             --WHERE CREATE_DT_DAY = '2021-10-18'
         '''.format(tera_target_table_name)
curs.execute(my_sql)

pd.DataFrame(curs.fetchall(), columns=cols)

# Populate Partitioned TD Tbl Using Stored Temporary Tbl

In [73]:
tera_table_name = "LAL_DB_HIST_IN_NONPART"
tera_target_table_name = "LAL_DB_HIST_IN"

In [23]:
JDBC_ARGUMENTS = "LOGMECH=LDAP, CHARSET=UTF8,TYPE=FASTEXPORT, COLUMN_NAME=ON, MAYBENULL=ON"
DB = 'PRD_DB_CLIENT4D_DEV1'
conn = jaydebeapi.connect(
    jclassname="com.teradata.jdbc.TeraDriver",
    url="jdbc:teradata://{}/database={} , {}".format(TERADATA_HOST, DB, JDBC_ARGUMENTS),
    driver_args={"user": USERNAME, "password": PASSWORD},
    jars=['/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/tdgssconfig.jar',
          '/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/terajdbc4.jar']
)
curs = conn.cursor()

In [75]:
my_sql = '''INSERT INTO PRD_DB_CLIENT4D_DEV1.{}
            SELECT
            *    
            FROM PRD_DB_CLIENT4D_DEV1.{}
            WHERE CREATE_DT_DAY IS NOT NULL
         '''.format(tera_target_table_name, tera_table_name)

curs.execute(my_sql)

# Get Info about Row-Partitioned Table

In [24]:
curs.execute("SHOW TABLE PRD_DB_CLIENT4D_DEV1.{};".format(tera_target_table_name))
curs.fetchall()

[("CREATE MULTISET TABLE PRD_DB_CLIENT4D_DEV1.MA_CMDM_MA_DEAL ,NO FALLBACK ,\r     NO BEFORE JOURNAL,\r     NO AFTER JOURNAL,\r     CHECKSUM = DEFAULT,\r     DEFAULT MERGEBLOCKRATIO,\r     MAP = TD_MAP2\r     (\r      ROW_ID INTEGER,\r      INN VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      KPP VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      HOST_APPL_ID VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      HOST_CRTR_EMP_ID VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      HOST_VKS_EMP_ID VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      HOST_CUST_ORG_ID VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      APPL_SALE_MTHD_TYPE_NAME VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      CREATE_DT TIMESTAMP(6) FORMAT 'YYYY-MM-DDBHH:MI:SS',\r      PLAN_COMPLETE_DT DATE FORMAT 'YYYY-MM-DD',\r      COMPLETE_DT DATE FORMAT 'YYYY-MM-DD',\r      HOST_PROD_ID VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      PROD_TYPE_NAME VA

In [113]:
query = \
'''
(
SELECT
* 
FROM dbc.indexconstraints
WHERE 1=1
AND databasename = '{}'
AND tablename = '{}'
AND constrainttype = 'Q'
) as t
'''.format(DB, tera_target_table_name)

sp.get_teradata(db, query).show()

,DatabaseName,TableName,IndexName,IndexNumber,ConstraintType,ConstraintText,ConstraintCollation,CollationName,CreatorName,CreateTimeStamp,CharSetID,SessionMode,ResolvedCurrent_Date,ResolvedCurrent_TimeStamp
0,PRD_DB_CLIENT4D_DEV1 ...,LAL_DB_HIST_IN ...,None,1,Q,CHECK ((RANGE_N(CREATE_DT_DAY BETWEEN DATE '2...,U,None,Ektov1-AV ...,2022-02-17 21:14:22,None,None,None,None


# ALTER PARTITIONS

In [114]:
last_part_lst

['2022-02-01',
 '2022-01-31',
 '2021-12-24',
 '2021-12-23',
 '2021-12-06',
 '2021-11-29',
 '2021-11-19',
 '2021-11-18',
 '2021-11-15',
 '2021-10-18']

In [118]:
query = \
'''
ALTER TABLE {}.{}
MODIFY
DROP RANGE BETWEEN DATE '2019-01-01' AND '2021-11-01'
--ADD RANGE BETWEEN DATE '2019-01-01' AND '2021-11-01'
--EACH INTERVAL '1' DAY
WITH DELETE;
'''.format(DB, tera_target_table_name)

curs.execute(query)

# Tranzact between Teradata and Hive DWH

In [120]:
cols = \
['INN',
 'MODEL_NAME',
 'HYPOTHESIS',
 'CLIENT_TYPE',
 'ID_SCEN',
 'CREATE_DT',
 'ONLY_SB',
 'DEAL_EVENT_DT',
 'MODEL_TYPE',
 'TARGET',
 'CREATE_DT_DAY',
 'ROW_ID']

DATABASE_NAME = tera_target_table_name

In [121]:
my_sql = '''(SELECT 
                 {cols}
             FROM {db}.{tbl} SAMPLE 100) as t
         '''.format(db=DB, tbl = DATABASE_NAME, cols = ", ".join(cols) )

sdf = sp.get_teradata(db, my_sql)

In [ ]:
sdf.show()

In [36]:
conn_schema = 'sbx_t_team_cvm'
table_name = DATABASE_NAME
hive.sql("drop table if exists {schema}.{tbl} purge".format(schema=conn_schema, tbl=table_name))
insert = ', '.join(["{} {}".format(col, _type) for col, _type in sdf.dtypes if col.lower()])

hive.sql('''create table {schema}.{tbl} (
                                         {fields}
                                            )
         '''.format(schema=conn_schema,
                    tbl=table_name,
                    fields=insert)
        )

DataFrame[]

In [37]:
my_sql = '''(SELECT 
                 {cols}
             FROM {db}.{tbl}) as t
         '''.format(db=DB, tbl = DATABASE_NAME, cols = ", ".join(cols) )
sdf = sp.get_teradata(db, my_sql, BATCH_SIZE=100000)

In [38]:
part_tbl = "tmp_"+table_name
sdf.registerTempTable(part_tbl)

In [39]:
%%time
hive.sql("""
insert into table {schema}.{tbl}
select * from {tmp_tbl}
""".format(schema=conn_schema,
           tbl=table_name,
           tmp_tbl=part_tbl)
        )

CPU times: user 272 ms, sys: 207 ms, total: 479 ms
Wall time: 21min 53s


DataFrame[]